# Домашнее задание №2

**

## Часть 1. ML workflow (**всего 5 баллов**)

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from itertools import product

### Загрузим данные для работы. 

In [3]:
df = pd.read_csv("winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Будем решать задачу регрессии: необходимо предсказать качество вина на основе его характеристик

### Шаг 1.  (**0.2 балла**)
Создайте матрицу X объект-признак и целевой вектор y ("quality")

In [4]:
df.dtypes

fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

In [5]:
X = df.iloc[:,:-1]
y = df['quality']

### Шаг 2. (**0.2 балла**)
Разбейте данные на train и test (доля тестовых данных - 30%).

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Шаг 3. (**0.2 балла**)
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [41]:
l_reg = LinearRegression()
l_reg.fit(X_train, y_train)
y_pred_test = l_reg.predict(X_test)
y_pred_train = l_reg.predict(X_train)

### Шаг 4. (**0.4 балла**)
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

In [54]:
print(f'''MSE loss: 
mse_train={mean_squared_error(y_train, y_pred_train)}
mse_test={mean_squared_error(y_test, y_pred_test)}
R2_train={r2_score(y_train,y_pred_train)}
R2_test={r2_score(y_test, y_pred_test)}
''')

MSE loss: 
mse_train=0.39116504963302506
mse_test=0.48491526880549135
R2_train=0.3676323607536507
R2_test=0.3220437514697857



### Шаг 5. (**0.5 балла**)
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

In [55]:
num_splits = 5
r2_mean = cross_val_score(estimator=l_reg, X=X, y=y, cv=num_splits, scoring='r2').mean()
print(f"""Cреднее качество (r2) ={r2_mean} """)

Cреднее качество (r2) =0.29004162884219614 


### Шаг 6.  (**0.5 балла**)
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1. 

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

In [44]:
from sklearn.linear_model import Lasso

In [45]:
lasso = Lasso()
n_alphas = 1.1/0.1
alphas =  np.linspace(0.1, 1.1, n_alphas)
params = {'alpha':alphas}
print(params)
cv = GridSearchCV(lasso,
                  params,
                  scoring='r2',
                  cv=num_splits
                 )
cv.fit(X_train, y_train)

print(cv.best_params_)

{'alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1])}
{'alpha': 0.1}


### Шаг 7.  (**0.5 балла**)
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [46]:
#your code here
print(f'''Лучшая модель - {cv.best_estimator_}
Наилучшее качество {cv.best_score_}
''')

Лучшая модель - Lasso(alpha=0.1)
Наилучшее качество 0.23086580138243312



### Шаг 8.  (**0.5 балла**)

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [47]:
lasso_cv = Lasso(alpha=0.1)
lasso_cv.fit(X_train, y_train)
y_pred_t = lasso_cv.predict(X_test) #делаем предсказание
R2_test_n= r2_score(y_test, y_pred_t)
print('r2-score на тесте:', R2_test_n)

r2-score на тесте: 0.20900374880379835


### Шаг 9.  (**0.5 балла**)

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [48]:
from sklearn.preprocessing import StandardScaler
pipe = Pipeline([('polynomial', PolynomialFeatures(degree = 2)),('model_', LinearRegression())]) # pipeline, состоящий из добавления полиномиальных признаков степени 2
r2_mean_pipe = cross_val_score(estimator=pipe, X=X, y=y, cv=num_splits, scoring='r2').mean()
print(f""" r2-score модели c полиномиальными признаками : {r2_mean_pipe} """)

 r2-score модели c полиномиальными признаками : 0.2300961676836561 


### Шаг 10.  (**0.5 балла**)
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [56]:
pipe.fit(X_train, y_train) #обучаем
pipe_train = pipe.predict(X_train) #предсказание для  train 
pipe_test = pipe.predict(X_test) #предсказание для test
print(f'''MSE loss: 
mse_train={mean_squared_error(y_train, pipe_train)}
mse_test={mean_squared_error(y_test, pipe_test)}
R2_train={r2_score(y_train,pipe_train)}
R2_test={r2_score(y_test, pipe_test)}
''')

MSE loss: 
mse_train=0.44550776322765134
mse_test=0.5742498128654916
R2_train=0.27978051013889427
R2_test=0.19714582341677445



### Сделайте выводы. Для этого ответьте на вопросы: (**1 балл**)

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

#### 1) Первая модель показала, что среднее качество (r2) =0.29, (R2_train=0.37, R2_test=0.32) , коэффициент детерминации небольшой, модель исходная модель не показала хорошее качество
#### Модель является переобученной,  mse_train=0.39 существенно ниже, чем mse_test=0.48. Среднеквадратическая ошибка на тренировочных данных значительно больше, чем на тренировочных.

#### 2) Нет, r2-score на тестовых данных лишь ухудшился до 0.21 при  L1-регуляризации

#### 3) Добавление полиномов второй степени не помогло улучшить качество модели. Средний r2-score модели c полиномиальными признаками : 0.23 (R2_train=0.28, R2_test=0.2). 
#### Добавление новых признаков повлияло на переобучение лишь усилила проблему переобучения, увеличив разрыв между mse_train=0.45 и mse_test=0.57.

In [58]:
#дополнение ко второму пункту : проблема переобучения лишь усугубилась
mean_squared_error(y_test, cv.predict(X_test)), mean_squared_error(y_train, cv.predict(X_train))


(0.5657683082123666, 0.4685439959972928)

### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы.

При улучшении качества r2 на 0.1-0.2 +1 балл, при большем улучшении +2 балла (дополнительно к 5 баллам за основную часть).

Можно немного улучшить качество, подобрав при L1-регуляризации alpha поменьше

In [79]:
alphas_2 =  np.linspace(0.0001, 0.1, n_alphas)
params_2 = {'alpha':alphas_2}
cv_2 = GridSearchCV(lasso,
                  params_2,
                  scoring='r2',
                  cv=num_splits
                 )
cv_2.fit(X_train, y_train)

print(cv_2.best_params_)

{'alpha': 0.0001}


In [80]:
print(f'''Лучшая модель - {cv_2.best_estimator_}
Наилучшее качество {cv_2.best_score_}
''')

Лучшая модель - Lasso(alpha=0.0001)
Наилучшее качество 0.34405602912718425



In [81]:
lasso_cv_2 = Lasso(alpha=0.0001)
lasso_cv_2.fit(X_train, y_train)
y_pred_t_2 = lasso_cv_2.predict(X_test)
R2_test_n_2= r2_score(y_test, y_pred_t_2)
print('r2-score на тесте:', R2_test_n_2)

r2-score на тесте: 0.3206615804398373


## Часть 2. Target encoding (**всего 5 баллов**)

В этом части домашнего задания вы будете работать с выборкой `1C`. Вам нужно посчитать счетчики для `item_id` четырьмя способами:

    1) При помощи KFold схемы;  
    2) При помощи Leave-one-out схемы;
    3) При помощи smoothing схемы;
    4) При помощи expanding mean схемы.

### Подготовка данных

In [82]:
sales = pd.read_csv('sales_train.csv.gz')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [83]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

### Mean encodings без регуляризации

После проделанной технической работы, мы готовы посчитать счетчики для переменной `item_id`. 

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.

#### Способ 1

In [84]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621791


#### Способ 2

In [85]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621791


###  KFold схема (**1.25 балла**)

Необходимо реализовать Kfold схему с пятью фолдами. Используйте KFold(5) из sklearn.model_selection. 

1. Разбейте данные на 5 фолдов при помощи `sklearn.model_selection.KFold` с параметром `shuffle=False`.
2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по `item_id` и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на **Способ 1** из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях. 

In [167]:
from sklearn.model_selection import KFold
num_splits = 5
kf = KFold(shuffle=False, n_splits = num_splits)

In [168]:
for train, test in kf.split(all_data):#получаем 4 обучающих фолда для подсчета
    print(len(train), len(test)) 

8731080 2182770
8731080 2182770
8731080 2182770
8731080 2182770
8731080 2182770


In [169]:
all_data.loc[all_data.index[0],'item_target_enc'] #первая строка таблицы all_data, столбец item_target_enc

0.3343

In [170]:
for train, test in kf.split(all_data):
    X_train = all_data.iloc[train] 
    X_train_target_mean  = X_train.groupby("item_id").target.mean() #Calculate a mapping
    all_data.loc[all_data.index[test],'item_target_enc'] = all_data.loc[all_data.index[test],'item_id'].map(X_train_target_mean)
    #применение функции к каждому элементу итерируемого объекта в test
    print(all_data.loc[all_data.index[test],'item_target_enc']) #проверяем, все ли работает так, как надо

139255          NaN
141495     0.048523
144968     0.142424
142661     0.030303
138947     0.894020
             ...   
2215941    0.101779
2213017    0.281195
2211991    0.759582
2212003    0.368177
2217092    0.084211
Name: item_target_enc, Length: 2182770, dtype: float64
2213024    0.121429
2218980    0.069149
2217428    0.082090
2218443    0.145602
2214603    0.137529
             ...   
4473029    0.071599
4480235    0.035842
4474764    0.115473
4473039    0.116844
4479664    0.099559
Name: item_target_enc, Length: 2182770, dtype: float64
4479663    0.080552
4479649    0.073384
4479650    0.067134
4479651    0.061060
4477061    0.094551
             ...   
6704022    0.256080
6704032    0.097826
6704031    0.024390
6701177    0.231568
6704987    0.028681
Name: item_target_enc, Length: 2182770, dtype: float64
6701512    0.057534
6701573    0.179245
6704465    0.019417
6704673    0.161538
6701571    0.320620
             ...   
8510008    6.700997
8508842    0.731047
8510609    0.32

In [171]:
all_data['item_target_enc'].fillna(0.3343, inplace=True) #заполняем пропуски

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.41645907127988024


Ожидаемый ответ 0.4165

### Leave-one-out схема (**1.25 балла**)

Необходимо реализовать leave-one-out схему . Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.
2. Вычесть таргет конкретного объекта и разделить результат на `n_objects - 1`. 

Заметим, что пункт `1.` следует сделать для всех объектов. Также заметим, что пункт `2.` может быть реализован без циклов `for`.

Здесь может оказаться полезной функция .transform из **Способа 2** из примера.

In [172]:
total_target = all_data['item_id'].map(all_data.groupby('item_id')['target'].sum()) #суммарный таргет по всем объектам
total_target

139255         1.0
141495        42.0
144968        84.0
142661        12.0
138947      2092.0
             ...  
10768834     699.0
10769024      75.0
10769690     493.0
10771216     348.0
10770511    1320.0
Name: item_id, Length: 10913850, dtype: float64

In [173]:
dif_target = (total_target - all_data['target']) #Вычитаем таргет конкретного объекта
n_objects = all_data['item_id'].map(all_data.groupby('item_id')['target'].count()) #количество объектов
all_data['item_target_enc'] = dif_target / (n_objects - 1) 
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.480384831129305


Ожидаемый ответ 0.4803

### Smoothing (**1.25 балла**)

Необходимо реализовать smoothing с $\alpha = 100$. Используйте формулу:

$\frac{mean(target) \cdot nrows + globalmean \cdot \alpha }{nrows + \alpha}$,

где $globalmean=0.3343$. Заметим, что `nrows` - это количество объектов, принадлежащих конктертной категории, а не количество строк в датасете.

In [174]:
alpha = 100
globalmean = 0.3343

mean_of_target = all_data.groupby('item_id')['target'].mean() #среднее таргета
nrows = all_data.groupby('item_id')['target'].count() #количество объектов, принадлежащих конктертной категории,
smoothing = (mean_of_target*nrows + globalmean*alpha) / (nrows + alpha) #формула
all_data['item_target_enc'] = all_data['item_id'].map(smoothing) #применяем функцию

#считаем корреляцию
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4818198797097282


Ожидаемый ответ 0.4818

### Expanding mean схема (**1.25 балла**)

Необходимо реализовать *expanding mean* схему. Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки $m$ использовать строки от $0$ до $m-1$. Вам будет необходимо воспользоваться pandas функциями [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) и [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html).

In [175]:
cumulative_sum= all_data.groupby('item_id')['target'].cumsum() - all_data['target'] #кумулятивная сумма элементов 
cumulative_count= all_data.groupby('item_id').cumcount() #кумулятивный подсчет строк 
all_data['item_target_enc'] = cumulative_sum/cumulative_count # применяем схему expanding mean 
all_data['item_target_enc'].fillna(0.3343, inplace=True) # заполняем пропуски 

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.5025245211081701


Ожидаемый ответ 0.5025